In [13]:
from glob import glob
import numpy as np

from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader

import torch
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable

from Infrastructure import CaptchaDataset, CNN, validate_model

import logging
logging.basicConfig(level=logging.DEBUG)

In [4]:
CUDA = False
if torch.cuda.is_available():
    CUDA = True
print(f'CUDA available: {CUDA}')

CUDA available: False


In [5]:
file_locations = glob('./Captchas/*')
captcha_names = [file.split('/')[-1].split('.')[0] for file in file_locations]
print( f'identified {len(file_locations)} images' )

identified 113062 images


In [6]:
# Unique characters is global -- expect only 60 out
unique_characters = [*set(char for name in captcha_names for char in name)]

print( f'{len(unique_characters)} unique characters: {unique_characters}' )

60 unique characters: ['r', 'K', 'V', '7', '3', '6', 'k', 'v', 'b', 'D', 'j', 'Q', 'g', 'h', 'R', 'd', 'N', 'O', 'x', 'B', 'c', 'S', '9', 'I', 'w', '8', 's', 'G', 'p', 'u', 'C', 'Y', 'q', 'i', '5', 'a', 'M', 'P', 'X', 'm', 'n', 'L', 'l', 'F', 'W', 'Z', 't', 'E', 'A', '1', 'J', '4', 'U', 'z', '2', 'y', 'T', 'H', 'f', 'e']


In [7]:
# Split training/test data
train_files, test_files = train_test_split(file_locations[0:3_000], test_size = .2)
print(f'Split dataset into 80:20 train/test of sizes {len(train_files)},{len(test_files)}.')

Split dataset into 80:20 train/test of sizes 2400,600.


In [8]:
# Load in training files.
train = CaptchaDataset.from_dir(train_files)#[0:128])

print(f'{train.X.shape}\nSample of images in format 40px x 150px x 3 RGB channels, of type {type(train.X[0][0][0][0])}')

# Instantiate dataloader (the iterable that provides batches for gradient descent.)
dl = DataLoader(train, \
    64, # Fetch 4 samples per batch
    shuffle=True, num_workers=2)

(2400, 40, 150, 3)
Sample of images in format 40px x 150px x 3 RGB channels, of type <class 'numpy.float32'>


In [9]:
# Test out the iterable.
dataiter = iter(dl)
images, label_array, labels = next(dataiter)
print(f'Each batch has a dataset of shape {images.shape} and a corresponding set of {label_array.shape} labels.')

Each batch has a dataset of shape torch.Size([64, 3, 40, 150]) and a corresponding set of torch.Size([64, 5, 62]) labels.


In [10]:
# Instantiate all our stuff
net = CNN()
if CUDA: net.cuda()
criterion = nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [11]:
for epoch in range(20):
    print(f'Starting epoch {epoch}')

    running_loss = 0.0
    for i,(images, label_array, labels) in enumerate(dl,0):
        
        # Zero param grads
        optimizer.zero_grad()

        if CUDA: images = Variable(images).cuda()
        if CUDA: label_array = Variable(label_array).cuda()

        # Forward
        prediction = net(images)
        
        # Calculate loss
        loss = criterion(prediction.reshape(prediction.shape[0],5,62), label_array)
        # Backpropagate
        loss.backward()
        # Step optimizer
        optimizer.step()

        # print stats
        running_loss += loss.item()
        interval = len(train_files) // 64 // 10
        if i % interval == interval-1: print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / interval:.3e}')

print('Finished Training')

Starting epoch 0
[1,     3] loss: 3.979e-01


KeyboardInterrupt: 

In [14]:
tl = DataLoader(CaptchaDataset.from_dir(test_files), \
    4, # Fetch 4 samples per batch
    shuffle=True, num_workers=2)

validate_model(net, tl)

UnboundLocalError: local variable 'captcha' referenced before assignment